# Convection cases
## Test case description
For a description of the test cases and their setup, please refer to the preprint/paper.
## Parameter and error metric explanation
Mandatory and optional data columns for the CSV files providing the plot data. See the repository `README.md`
for general syntax information.

| column name | description | required |
| :-          | :-          | :-       |
| `solver` | solver name, e.g. *interFoam* | yes |  
| `resolution` | mesh resolution as number of cells per spatial direction | yes |  
| `l1_shape_error` | See paper | yes |  
| `volume_error` | Global, absolute difference between initial phase volume and phase volume at end time | no |
| `boundedness_error` | Maximum value how much numerical alpha either overshoots 1 or undershoots 0 | no |
| `min_alpha` | Minimum alpha value at end time | no |
| `max_alpha` | Maximum alpha value at end time | no |


## Usage
The notebook is prepared such that you can just hit `Kernel` ->
`Restart & Run All`. This plots the _L1 shape error_ over _resolution_ for all solvers present in the data directories.

### Quick customization
Set the list `test_cases` below to the test cases for which you want to create a comparison plot. Currently, the list contains the three advection test cases reported in the paper.  
The list `data_columns` has to contain the entry `resolution`. In addition, it has to contain one or more names of error metrics. Preset is the
`l1_shape_error`.  
Per default, all solvers for which data is available are added to each plot. If you want to compare a specific set of solvers, remove the `'all'` entry from `solvers` below and list the solvers you want to compare, e.g.
`solvers = ['interFoam', 'Fluent']`. If you prescribe a solver subset and a solver is missing in the plots, make sure that its spelling matches the entries in the CSV file exactly.

In [ ]:
test_cases = ['discInDiagonalFlow2D',
              'discInReversedVortexFlow2D',
              'sphereInReversedVortexFlow3D']
data_columns = ['resolution', 'l1_shape_error']
index_columns = ['solver']

# Path to common data directory
directory = 'data/convection/'

# Solver selection
solvers = ['all']

# Assign better formatted names to errors used as axis label
formatted_errors = {'l1_shape_error' : '$L_1$ shape error'}

In [ ]:
#--- Imports ---
from shared_functions import *
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update(latex_settings)

def create_plots(test_case, index_columns, data_columns, data_path,
                 formatted_errors, solver_subset=['all']):
    path = os.path.join(data_path, test_case)
    data_files = [filename for filename in os.listdir(path) if 'csv' in filename]
    
    df = read_solver_data(data_files, index_columns, data_columns, data_dir=path, solver_subset=solver_subset)

    errors_to_plot = list(data_columns)
    errors_to_plot.remove('resolution')

    solvers = list(df.index.unique(level='solver'))
    solver_marker, solver_line = assign_styles_to_solvers(solvers)

    for error in errors_to_plot:
        fig, ax = plt.subplots(figsize=figureSize, dpi=DPI)
     
        for solver, plotdf in df.groupby(level="solver"):
            ax.loglog(plotdf["resolution"], plotdf[error],
                      label=solver, marker=solver_marker[solver], linestyle='None', markerfacecolor='None')       
        
        # Set x-ticks to used resolutions
        resolutions = list(np.unique(df['resolution']))
        ax.set_xticks(resolutions)
        ax.set_xticklabels([str(res) for res in resolutions])
        ax.set_xticklabels([], minor=True)
    
        # Set axis labels
        error_name = error
        if error in formatted_errors.keys():
            error_name = formatted_errors[error]
        ax.set_ylabel(error_name, fontsize=plotfontsize)
        ax.set_xlabel("resolution", fontsize=plotfontsize)
        ax.tick_params(axis='both', labelsize=plotfontsize)
        
        ax.grid(which="major")
        ax.legend(fontsize=plotfontsize)

        if not os.path.isdir(figure_directory):
            os.mkdir(figure_directory)
        fig.savefig(os.path.join(figure_directory, test_case + "_" + error + figure_suffix),
                  bbox_inches="tight")
    
        # Display title in the notebook
        ax.set_title(test_case, fontsize=plotfontsize)

# Create Plots for all convection test cases
for case in test_cases:
    create_plots(case, index_columns, data_columns, directory, formatted_errors, solver_subset=solvers)